<a href="https://colab.research.google.com/github/ohadbarr1/Thesis/blob/main/Chronos_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and initializations

Install chronos

In [ ]:
pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-hb9s4zsx
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-hb9s4zsx
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit eb7bdfc047de3e7af972b4ee7cf23a7968b7daa3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12

## **Import libraries**

In [ ]:
# Import relevant libraries
import pandas as pd
import numpy as np
from google.colab import files
from datetime import timedelta

# Data preprocessing
from sklearn.preprocessing import MinMaxScaler

# Gather financial data
import yfinance as yf

# Plots
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Models
import torch
from sklearn.model_selection import train_test_split
from chronos import ChronosPipeline

## Initializing pipeline

In [ ]:
# Use the Chronos-t5-base model, 200m parameters
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-base",
  device_map="cuda",
  torch_dtype=torch.bfloat16,
)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

### Gather VIX Data

In [ ]:
# Data fetching and preprocessing
tickers = ['^VIX9D', '^VIX', '^VIX3M', '^VIX6M']
data = yf.download(tickers, start="2011-01-03")['Close']
# Reordering columns and renaming them according to your format
data = data[['^VIX', '^VIX3M', '^VIX6M', '^VIX9D']]
data.columns = ['VIX', 'VIX3M', 'VIX6M', 'VIX9D']
# Resetting index to make 'Date' a column
data.reset_index(inplace=True)

data



[*********************100%%**********************]  4 of 4 completed


,Date,VIX,VIX3M,VIX6M,VIX9D
0,2011-01-03,17.610001,20.620001,23.400000,16.040001
1,2011-01-04,17.379999,20.610001,23.190001,16.059999
2,2011-01-05,17.020000,20.049999,22.780001,15.570000
3,2011-01-06,17.400000,20.350000,22.870001,15.710000
4,2011-01-07,17.139999,20.290001,22.920000,15.010000
...,...,...,...,...,...
3419,2024-08-06,27.709999,26.760000,25.559999,28.930000
3420,2024-08-07,27.850000,27.010000,25.750000,29.260000
3421,2024-08-08,23.790001,24.129999,23.500000,23.900000
3422,2024-08-09,20.370001,21.180000,21.430000,20.059999


In [ ]:
VIX9D = data[['Date','VIX9D']]
VIX = data[['Date','VIX']]
VIX3M = data[['Date','VIX3M']]
VIX6M = data[['Date','VIX6M']]


VIX Data

In [ ]:
# Avoid SettingWithCopyWarning by using .loc
VIX.loc[:, 'Date'] = pd.to_datetime(VIX['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast_1', 'Forecast_2', 'Forecast_3', 'Forecast_4', 'Forecast_5', 'Forecast_Avg'])

# Number of days in three days (approx.)
three_days = timedelta(days=3)

# Loop over the DataFrame in steps of three days
for i in range(1000, len(VIX), 3):  # 3 trading days interval
    # Define the current date
    current_date = VIX['Date'].iloc[i]
    end_date = current_date + three_days

    # Use all data up to the current date as context
    context_data = VIX[VIX['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_three_days = VIX[(VIX['Date'] > current_date) & (VIX['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Run the model 5 times
    for run in range(5):
        # Make a prediction for the next three days
        prediction_length = min(len(actual_next_three_days), 3)  # Predicting 3 days or fewer if the actual data is less
        if prediction_length > 0:  # Ensure there's something to predict
            forecast = pipeline.predict(context, prediction_length)
            if forecast is not None and len(forecast) > 0:
                forecast = forecast.squeeze().cpu().numpy()

                # Ensure forecast is 1-dimensional
                if forecast.ndim > 1:
                    forecast = forecast.flatten()

                # Store the forecast
                forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_three_days['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_three_days['VIX'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_1': forecasts[0],
            'Forecast_2': forecasts[1],
            'Forecast_3': forecasts[2],
            'Forecast_4': forecasts[3],
            'Forecast_5': forecasts[4],
            'Forecast_Avg': forecast_avg
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX Values (Averaged Forecast - 5 Iterations, 3-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()

# prompt: convert to csv and download results_df and call it "VIX.csv"

# Assuming 'results' is your DataFrame
results.to_csv('VIX_pred.csv', index=False)
files.download('VIX_pred.csv')


<ipython-input-8-e4765311d243>:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, result_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

VIX 9D

In [ ]:
# Avoid SettingWithCopyWarning by using .loc
VIX9D.loc[:, 'Date'] = pd.to_datetime(VIX9D['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast_1', 'Forecast_2', 'Forecast_3', 'Forecast_4', 'Forecast_5', 'Forecast_Avg'])

# Number of days in three days (approx.)
three_days = timedelta(days=3)

# Loop over the DataFrame in steps of three days
for i in range(1000, len(VIX9D), 3):  # 3 trading days interval
    # Define the current date
    current_date = VIX9D['Date'].iloc[i]
    end_date = current_date + three_days

    # Use all data up to the current date as context
    context_data = VIX9D[VIX9D['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_three_days = VIX9D[(VIX9D['Date'] > current_date) & (VIX9D['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX9D"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Run the model 5 times
    for run in range(5):
        # Make a prediction for the next three days
        prediction_length = min(len(actual_next_three_days), 3)  # Predicting 3 days or fewer if the actual data is less
        if prediction_length > 0:  # Ensure there's something to predict
            forecast = pipeline.predict(context, prediction_length)
            if forecast is not None and len(forecast) > 0:
                forecast = forecast.squeeze().cpu().numpy()

                # Ensure forecast is 1-dimensional
                if forecast.ndim > 1:
                    forecast = forecast.flatten()

                # Store the forecast
                forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_three_days['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_three_days['VIX9D'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_1': forecasts[0],
            'Forecast_2': forecasts[1],
            'Forecast_3': forecasts[2],
            'Forecast_4': forecasts[3],
            'Forecast_5': forecasts[4],
            'Forecast_Avg': forecast_avg
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX9D Values (Averaged Forecast - 5 Iterations, 3-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX9D Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()

# prompt: convert to csv and download results_df and call it "VIX9D.csv"

# Assuming 'results' is your DataFrame
results.to_csv('VIX9D_pred.csv', index=False)
files.download('VIX9D_pred.csv')


<ipython-input-9-c91d50daaaa3>:73: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

VIX3M

In [ ]:
# Avoid SettingWithCopyWarning by using .loc
VIX3M.loc[:, 'Date'] = pd.to_datetime(VIX3M['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast_1', 'Forecast_2', 'Forecast_3', 'Forecast_4', 'Forecast_5', 'Forecast_Avg'])

# Number of days in three days (approx.)
three_days = timedelta(days=3)

# Loop over the DataFrame in steps of three days
for i in range(1000, len(VIX3M), 3):  # 3 trading days interval
    # Define the current date
    current_date = VIX3M['Date'].iloc[i]
    end_date = current_date + three_days

    # Use all data up to the current date as context
    context_data = VIX3M[VIX3M['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_three_days = VIX3M[(VIX3M['Date'] > current_date) & (VIX3M['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX3M"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Run the model 5 times
    for run in range(5):
        # Make a prediction for the next three days
        prediction_length = min(len(actual_next_three_days), 3)  # Predicting 3 days or fewer if the actual data is less
        if prediction_length > 0:  # Ensure there's something to predict
            forecast = pipeline.predict(context, prediction_length)
            if forecast is not None and len(forecast) > 0:
                forecast = forecast.squeeze().cpu().numpy()

                # Ensure forecast is 1-dimensional
                if forecast.ndim > 1:
                    forecast = forecast.flatten()

                # Store the forecast
                forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_three_days['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_three_days['VIX3M'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_1': forecasts[0],
            'Forecast_2': forecasts[1],
            'Forecast_3': forecasts[2],
            'Forecast_4': forecasts[3],
            'Forecast_5': forecasts[4],
            'Forecast_Avg': forecast_avg
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX3M Values (Averaged Forecast - 5 Iterations, 3-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX3M Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()

# prompt: convert to csv and download results_df and call it "VIX3M.csv"

# Assuming 'results' is your DataFrame
results.to_csv('VIX3M.csv', index=False)
files.download('VIX3M_pred.csv')


<ipython-input-10-7c77f477b905>:73: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



FileNotFoundError: Cannot find file: VIX3M_pred.csv

In [ ]:
# Assuming 'results' is your DataFrame
results.to_csv('VIX3M_pred.csv', index=False)
files.download('VIX3M_pred.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

VIX6M

In [ ]:
# Avoid SettingWithCopyWarning by using .loc
VIX6M.loc[:, 'Date'] = pd.to_datetime(VIX6M['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast_1', 'Forecast_2', 'Forecast_3', 'Forecast_4', 'Forecast_5', 'Forecast_Avg'])

# Number of days in three days (approx.)
three_days = timedelta(days=3)

# Loop over the DataFrame in steps of three days
for i in range(1000, len(VIX6M), 3):  # 3 trading days interval
    # Define the current date
    current_date = VIX6M['Date'].iloc[i]
    end_date = current_date + three_days

    # Use all data up to the current date as context
    context_data = VIX6M[VIX6M['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_three_days = VIX6M[(VIX6M['Date'] > current_date) & (VIX6M['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX6M"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Run the model 5 times
    for run in range(5):
        # Make a prediction for the next three days
        prediction_length = min(len(actual_next_three_days), 3)  # Predicting 3 days or fewer if the actual data is less
        if prediction_length > 0:  # Ensure there's something to predict
            forecast = pipeline.predict(context, prediction_length)
            if forecast is not None and len(forecast) > 0:
                forecast = forecast.squeeze().cpu().numpy()

                # Ensure forecast is 1-dimensional
                if forecast.ndim > 1:
                    forecast = forecast.flatten()

                # Store the forecast
                forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_three_days['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_three_days['VIX6M'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_1': forecasts[0],
            'Forecast_2': forecasts[1],
            'Forecast_3': forecasts[2],
            'Forecast_4': forecasts[3],
            'Forecast_5': forecasts[4],
            'Forecast_Avg': forecast_avg
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX6M Values (Averaged Forecast - 5 Iterations, 3-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX6M Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()

# prompt: convert to csv and download results_df and call it "VIX6M.csv"

# Assuming 'results' is your DataFrame
results.to_csv('VIX6M_pred.csv', index=False)
files.download('VIX6M_pred.csv')


<ipython-input-12-91e5d793edda>:73: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### SPY predictions

In [ ]:
SPY = yf.download('SPY', start="2011-01-03")['Close']
SPY =  SPY.reset_index()
SPY

[*********************100%%**********************]  1 of 1 completed


,Date,Close
0,2011-01-03,127.050003
1,2011-01-04,126.980003
2,2011-01-05,127.639999
3,2011-01-06,127.389999
4,2011-01-07,127.139999
...,...,...
3422,2024-08-09,532.989990
3423,2024-08-12,533.270020
3424,2024-08-13,542.039978
3425,2024-08-14,543.750000


In [ ]:
import pandas as pd
import numpy as np
import torch
from datetime import timedelta
import yfinance as yf
import plotly.graph_objects as go

# Download SPY data and select the 'Close' column
SPY = yf.download('SPY', start="2011-01-03")['Close']
SPY = SPY.reset_index()

# Avoid SettingWithCopyWarning by using .loc
SPY.loc[:, 'Date'] = pd.to_datetime(SPY['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast_1', 'Forecast_2', 'Forecast_3', 'Forecast_4', 'Forecast_5', 'Forecast_Avg'])

# Number of days in three days (approx.)
interval = timedelta(days=2)

# Loop over the DataFrame in steps of five days
for i in range(1000, len(SPY), 2):  # 2 trading days interval
    # Define the current date
    current_date = SPY['Date'].iloc[i]
    end_date = current_date + interval

    # Use all data up to the current date as context
    context_data = SPY[SPY['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_days = SPY[(SPY['Date'] > current_date) & (SPY['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["Close"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Run the model 5 times
    for run in range(5):
        # Make a prediction for the next three days
        prediction_length = min(len(actual_next_days), 3)  # Predicting 3 days or fewer if the actual data is less
        if prediction_length > 0:  # Ensure there's something to predict
            forecast = pipeline.predict(context, prediction_length)
            if forecast is not None and len(forecast) > 0:
                forecast = forecast.squeeze().cpu().numpy()

                # Ensure forecast is 1-dimensional
                if forecast.ndim > 1:
                    forecast = forecast.flatten()

                # Store the forecast
                forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_days['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_days['Close'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_1': forecasts[0],
            'Forecast_2': forecasts[1],
            'Forecast_3': forecasts[2],
            'Forecast_4': forecasts[3],
            'Forecast_5': forecasts[4],
            'Forecast_Avg': forecast_avg
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual SPY Close Values (Averaged Forecast - 5 Iterations, 3-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="SPY Close Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()

# Save the results to CSV and download
results.to_csv('SPY_Close_Forecast.csv', index=False)
files.download('SPY_Close_Forecast.csv')


[*********************100%%**********************]  1 of 1 completed
<ipython-input-19-699bb5b19f40>:84: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>